In [2]:
# IMPORT MODULES AND LIBRARIES FOR GUIs
import tkinter as tk
from tkinter import ttk
from tkinter import E, W
import tkinter.messagebox as msgBox
from PIL import Image, ImageTk
import xlsxwriter as xl_write

In [3]:
# IMPORT MODULES AND LIBRARIES FOR GA
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import time
import random
import itertools
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.compose import make_column_transformer
from keras.models import load_model

In [4]:
# RECIPE PREDICTING GUI 
# GENETIC ALGORITM FUNCTIONS
# MAKING MATERIAL DICTIONARIES

monomer_list = {'wt_IPDA': 'wt_IPDA',
                'wt_HDDA': 'wt_HDDA',
                'wt_TMPTA': 'wt_TMPTA',
                'wt_TMPEOTA': 'wt_TMPEOTA',
                 'wt_DEM': 'wt_DEM',
                 'wt_DBM': 'wt_DBM',
                 'wt_DOM': 'wt_DOM',
                 'wt_IPDI': 'wt_IPDI'}


In [5]:
# Make Monomer Combinations

def get_monomers(monomer_ranges):
    monomer_combos = np.array(list(itertools.product(*[np.arange(
        #load_range[0], load_range[1])
        load_range[0], load_range[1], 0.01) 
        for load_range in monomer_ranges])))

    monomer_combos = [combo for combo in monomer_combos \
                    if sum(combo).round(5) <= 1]
    

    monomer_combos = df(np.vstack(monomer_combos), columns = \
                        ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA', 'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'])

    return monomer_combos.values
    

In [6]:
# GUESSING A RECIPE WITH SPECIFIED PERFORMANCE (with test case)

all_genes = [
    get_monomers(monomer_ranges = [[0.25, 0.30], #wt_IPDA (0.27)
                                   [0.00, 0.10], #wt_HDDA (0.04)
                                   [0.00, 0.01], #wt_TMPTA (0.00)
                                   [0.00, 0.01], #wt_TMPEOTA (0.00)
                                   [0.05, 0.10], #wt_DEM (0.06)
                                   [0.35, 0.45], #wt_DBM (0.39)
                                   [0.10, 0.15], #wt_DOM (0.12)
                                   [0.10, 0.15]] #wt_IPDI (0.12)
                )
    ]

# all_genes = [
#     get_monomers(monomer_ranges = [[0.25, 0.30], 
#                                    [0.00, 0.10], 
#                                    [0.00, 0.00], 
#                                    [0.00, 0.00], 
#                                    [0.05, 0.10], 
#                                    [0.35, 0.45], 
#                                    [0.10, 0.15], 
#                                    [0.10, 0.15]] 
#                 )
#     ]

## This is the geneset from which all recipes will be generated. 
## The generate_parent function will randomly select values from this dataset.

In [7]:
all_genes

[array([[0.25, 0.  , 0.  , ..., 0.35, 0.1 , 0.1 ],
        [0.25, 0.  , 0.  , ..., 0.35, 0.1 , 0.11],
        [0.25, 0.  , 0.  , ..., 0.35, 0.1 , 0.12],
        ...,
        [0.29, 0.09, 0.  , ..., 0.35, 0.11, 0.1 ],
        [0.29, 0.09, 0.  , ..., 0.36, 0.1 , 0.1 ],
        [0.29, 0.09, 0.  , ..., 0.35, 0.1 , 0.1 ]])]

In [8]:
type(all_genes)

list

In [9]:
length_all_genes = len(all_genes)
print(length_all_genes)

1


In [10]:
# Load Neural Network for continuous model

# continuous_model = load_model('kfold continuous network_all_SAPPER.h5')

continuous_model = load_model('Axalta_kfold_continuous_network_all_SAPPER.h5')

# these are the neural networks that will be used to predict the
# performance properties of the recipes generated

In [11]:
# Establish Transformers and Scalers

In [12]:
X_transformer = make_column_transformer(
        (MaxAbsScaler(),
        ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI']))

# This function scales and encodes the recipes to ensure they can be
# properly interpretted by the neural networks.
# (This is X_preprocessor in NN script)

In [13]:
X_scaler = pd.read_csv('scaler_AxaltaData_210318_clean_070821.csv', usecols = [
        'wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'])

# This scaler dataframe is added to all recipes to ensure all properties
# have the correct scaling.

In [14]:
Y_transformer = make_column_transformer(
        (MaxAbsScaler(),
         ['fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
        'gloss_1day', 'DOI_7d', 'gloss_7d',
        'Amtec_afterScratch', 'Amtec_afterReflow', 
        'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600']))

# This scales and encodes the target data.
# (This is Y_preprocessor in NN script.)

In [15]:
Y_scaler = pd.read_csv('scaler_AxaltaData_210318_clean_070821.csv', usecols = [
        'fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
        'gloss_1day', 'DOI_7d', 'gloss_7d',
        'Amtec_afterScratch', 'Amtec_afterReflow', 
        'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600'])

# This scaler dataframe is added to the target recipe to ensure all
# properties have the correct scaling.

In [16]:
# Genetic Algorithm Fitness Conditions

def fisher_2h_condition(predictions, target, limits):
    if limits == [0,0] and target['fisher_2h'].values != 0:
        if abs(predictions['fisher_2h'].values -
               target['fisher_2h'].values) <= 1:   # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['fisher_2h'].values <= limits[0]:
            return 1
        else:
            return 0

In [17]:
def fisher_24h_condition(predictions, target, limits):
    if limits == [0,0] and target['fisher_24h'].values != 0:
        if abs(predictions['fisher_24h'].values -
               target['fisher_24h'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['fisher_24h'].values <= limits[0]:
            return 1
        else:
            return 0

In [18]:
def fisher_7d_condition(predictions, target, limits):
    if limits == [0,0] and target['fisher_7d'].values != 0:
        if abs(predictions['fisher_7d'].values -
               target['fisher_7d'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['fisher_7d'].values <= limits[0]:
            return 1
        else:
            return 0

In [19]:
def DOI_1day_condition(predictions, target, limits):
    if limits == [0,0] and target['DOI_1day'].values != 0:
        if abs(predictions['DOI_1day'].values -
               target['DOI_1day'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['DOI_1day'].values <= limits[0]:
            return 1
        else:
            return 0

In [20]:
def gloss_1day_condition(predictions, target, limits):
    if limits == [0,0] and target['gloss_1day'].values != 0:
        if abs(predictions['gloss_1day'].values -
               target['gloss_1day'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['gloss_1day'].values <= limits[0]:
            return 1
        else:
            return 0

In [21]:
def DOI_7day_condition(predictions, target, limits):
    if limits == [0,0] and target['DOI_7day'].values != 0:
        if abs(predictions['DOI_7day'].values -
               target['DOI_7day'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['DOI_7day'].values <= limits[0]:
            return 1
        else:
            return 0

In [22]:
def gloss_7day_condition(predictions, target, limits):
    if limits == [0,0] and target['gloss_7day'].values != 0:
        if abs(predictions['gloss_7day'].values -
               target['gloss_7day'].values) <= 1:     # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['gloss_7day'].values <= limits[0]:
            return 1
        else:
            return 0

In [23]:
def Amtec_afterScratch_condition(predictions, target, limits):
    if limits == [0,0] and target['Amtec_afterScratch'].values != 0:
        if abs(predictions['Amtec_afterScratch'].values -
               target['Amtec_afterScratch'].values) <= 1:   # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['Amtec_afterScratch'].values <= limits[0]:
            return 1
        else:
            return 0

In [24]:
def Amtec_afterReflow_condition(predictions, target, limits):
    if limits == [0,0] and target['Amtec_afterReflow'].values != 0:
        if abs(predictions['Amtec_afterReflow'].values -
               target['Amtec_afterReflow'].values) <= 1:   # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['Amtec_afterReflow'].values <= limits[0]:
            return 1
        else:
            return 0

In [25]:
def Amtect_afterScratchRel6600_condition(predictions, target, limits):
    if limits == [0,0] and target['Amtect_afterScratchRel6600'].values != 0:
        if abs(predictions['Amtect_afterScratchRel6600'].values -
               target['Amtect_afterScratchRel6600'].values) <= 1:   # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['Amtect_afterScratchRel6600'].values <= limits[0]:
            return 1
        else:
            return 0

In [26]:
def Amtec_afterReflowRel6600_condition(predictions, target, limits):
    if limits == [0,0] and target['Amtec_afterReflowRel6600'].values != 0:
        if abs(predictions['Amtec_afterReflowRel6600'].values -
               target['Amtec_afterReflowRel6600'].values) <= 1:   # The 1 defines the allowable tolerance.
            return 1
        else:
            return 0
    else:
        if limits[1] <= predictions['Amtec_afterReflowRel6600'].values <= limits[0]:
            return 1
        else:
            return 0

In [27]:
conditions = [fisher_2h_condition, fisher_24h_condition,
              fisher_7d_condition, DOI_1day_condition,
              gloss_1day_condition, DOI_7day_condition,
              gloss_7day_condition, Amtec_afterScratch_condition,
              Amtec_afterReflow_condition, Amtect_afterScratchRel6600_condition, 
              Amtec_afterReflowRel6600_condition]

# This compiles the list of functions to be used in the fitness function
# to evaluate the recipes based on the users' upper and lower limits

In [28]:
# Define remainig helper functions:

In [29]:
def pre_processor(data):
#     data['Prom Chem'] = data['Prom'].map(promoter_chemistry)
#     data['Prom Interaction'] = data['Prom'].map(promoter_interaction_type)
#     data['Prom # Inter'] = data['Prom'].map(promoter_number_of_interactions)
#     data['Hydrophile'] = data['Surfactant'].map(surfactant_hydrophile)
#     data['Ionic Nature'] = data['Surfactant'].map(surfactant_ionic_nature)
#     data['Polymerizability'] = data['Surfactant'
#                                     ].map(surfactant_polymerizability)
#     data['# PG Units'] = data['Surfactant'
#                               ].map(surfactant_number_of_PG_units)
#     data['# EG Units'] = data['Surfactant'
#                               ].map(surfactant_number_of_EG_units)
#     data['Theo Tg'] = get_Tg(data)
#     # this adds characteristics of the materials to the dataset

    data = X_transformer.fit_transform(data.append(X_scaler, sort = 'false'))
    data = data[0:(len(data)-len(X_scaler)), :]
    return data

# This overall function processes the data so that it can be properly
# interpreted by the neural networks.
# This function requires the input data to be a dataframe.

In [30]:
def target_processor(data):
    data = Y_transformer.fit_transform(data.append(Y_scaler, sort = 'false'))
    data = data[0:(len(data)-len(Y_scaler)), :]
    return data

# this function processes the target values so they have the same scale
# and can be properly interpreted by the fitness function
# this function requires the input data to be a dataframe

In [31]:
def unscaler(data):
    if type(data) == pd.DataFrame:
        data['fisher_2h'] = (data['fisher_2h']*200).round(2)
        data['fisher_24h'] = (data['fisher_24h']*200).round(2)
        data['fisher_7d'] = (data['fisher_7d']*200).round(2)
        data['DOI_1d'] = (data['DOI_1d']*100).round(2)
        data['gloss_1d'] = (data['gloss_1d']*100).round(2)
        data['DOI_7d'] = (data['DOI_7d']*100).round(2)
        data['gloss_7d'] = (data['gloss_7d']*100).round(2)
        data['Amtec_afterScratch'] = (data['Amtec_afterScratch']*100).round(2)
        data['Amtec_afterReflow'] = (data['Amtec_afterReflow']*100).round(2)
        data['Amtect_afterScratchRel6600'] = (data['Amtect_afterScratchRel6600']*2).round(2)
        data['Amtec_afterReflowRel6600'] = (data['Amtec_afterReflowRel6600']*2).round(2)
        
    elif type(data) == np.ndarray:
        data[:,0] = (data[:,0]*200).round(2) # Fisher2h... not sure about 200 here. Or indexer.
        data[:,1] = (data[:,1]*200).round(2) # Fisher24h... not sure about 200 here. Or indexer.
        data[:,2] = (data[:,2]*200).round(2) # Fisher7d... not sure about 200 here. Or indexer.
        data[:,3] = (data[:,3]*100).round(2) # DOI_1d
        data[:,4] = (data[:,4]*100).round(2) # gloss_1d
        data[:,5] = (data[:,5]*100).round(2) # DOI_7d
        data[:,6] = (data[:,6]*100).round(2) # gloss_7d
        data[:,7] = (data[:,7]*100).round(2) # Amtec_afterScratch
        data[:,8] = (data[:,8]*100).round(2) # Amtec_afterReflow
        data[:,9] = (data[:,9]*2).round(2) # Amtect_afterScratchRel6600
        data[:,10] = (data[:,10]*2).round(2) # Amtec_afterReflowhRel6600
        
        data = df(data, columns = ['fisher_2h', 'fisher_24h', 'fisher_7d',
                                   'DOI_1d', 'gloss_1d', 'DOI_7d',
                                  'gloss_7d', 'Amtec_afterScratch',
                                  'Amtec_afterReflow', 'Amtect_afterScratchRel6600',
                                  'Amtec_afterReflowRel6600'])
    return data

# this function returns predicted data back to its original scale

In [32]:
def percent_scaler(loadings):
    column_list = ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
                   'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI']
    for column in column_list:
        loadings[column] = loadings[column]*100
    return loadings

# This function scales the percentage loadings to a 100 scale.
# This function requires data to be a dataframe.

In [33]:
def value_decoder(data):
    column_list = []
    for n in np.arange(0,len(data)):
        if sum(data[n,:]) == 0:
            column_list.append(0)
        for column in np.arange(0,len(data[0])):
            if data[n,column] != 0:
                column_list.append(column)
    return np.array(column_list)

# This function converts dummy variables back to their original values

In [34]:
def property_predictor(data):
    dataframe = unscaler(df(continuous_model.predict(pre_processor(data)),
                            columns = ['fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
                                       'gloss_1day', 'DOI_7d', 'gloss_7d',
                                       'Amtec_afterScratch', 'Amtec_afterReflow', 
                                       'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600'])).round(2)
    return dataframe

# This function makes property predictions based on the recipe which will
# be evaluated in the fitness function.

In [35]:
# Establish GA functions

def _generate_parent(gene_set):
    genes = df(index = [1])
    monomers = random.choice(gene_set[3])
    
    # genes['Prom'] = random.choice(gene_set[0])
    # genes['Prom Load'] = monomers[4]
    # genes['Surfactant'] = random.choice(gene_set[1])
    # genes['Surf Loading'] = random.choice(gene_set[2])
    
    genes['wt_IPDA'] = monomers[0]
    genes['wt_HDDA'] = monomers[1]
    genes['wt_TMPTA'] = monomers[2]
    genes['wt_TMPEOTA'] = monomers[3]
    genes['wt_DEM'] = monomers[4]
    genes['wt_DBM'] = monomers[5]
    genes['wt_DOM'] = monomers[6]
    genes['wt_IPDI'] = monomers[7]
    
    # genes['Theo % Solids'] = random.choice(gene_set[4])
    
    return genes
    
    # This function generates a dataframe of a polymer recipe to be later
    # evaluated in the fitness function and mutated if necessary


def _mutate(parent):
    child_genes = parent.rename(index = str, columns =
                                {'wt_IPDA': 0, 'wt_HDDA': 1,
                                 'wt_TMPTA': 2, 'wt_TMPEOTA': 3,
                                 'wt_DEM': 4, 'wt_DBM': 5,
                                 'wt_DOM': 6, 'wt_IPDI': 7})                        
    index = random.choice(child_genes.columns.values)
    if index == 0:
        new_gene, alternate = random.sample(all_genes[0], 2)
        if child_genes[index].values == new_gene:
            child_genes = child_genes.replace(child_genes[index].values,
                                              value = alternate)
        else:
            child_genes = child_genes.replace(child_genes[index].values,
                                              value = new_gene)
    #elif index == 1 or index == 4 or index == 5 or \
         #index == 6 or index == 7 or index == 8:
    elif index == 1 or index == 2 or index == 3 or \
         index == 4 or index == 5 or index == 6:
        new_mon = random.choice(all_genes[3])
        alternate_mon = random.choice(all_genes[3])
        if all(alternate_mon) == all(new_mon):
            alternate_mon = random.choice(all_genes[3])
        if child_genes[index].values == new_mon[0] or \
           child_genes[index].values == new_mon[1] or \
           child_genes[index].values == new_mon[2] or \
           child_genes[index].values == new_mon[3] or \
           child_genes[index].values == new_mon[4] or \
           child_genes[index].values == new_mon[5]:
            child_genes = child_genes.replace(child_genes[1].values,
                                              value = alternate_mon[4])
            child_genes = child_genes.replace(child_genes[4].values,
                                              value = alternate_mon[0])
            child_genes = child_genes.replace(child_genes[5].values,
                                              value = alternate_mon[1])
            child_genes = child_genes.replace(child_genes[6].values,
                                              value = alternate_mon[2])
            #child_genes = child_genes.replace(child_genes[7].values,
                                              #value = alternate_mon[3])
            #child_genes = child_genes.replace(child_genes[8].values,
                                              #value = alternate_mon[5])
        else:
            child_genes = child_genes.replace(child_genes[1].values,
                                              value = new_mon[4])
            child_genes = child_genes.replace(child_genes[4].values,
                                              value = new_mon[0])
            child_genes = child_genes.replace(child_genes[5].values,
                                              value = new_mon[1])
            child_genes = child_genes.replace(child_genes[6].values,
                                              value = new_mon[2])
            #child_genes = child_genes.replace(child_genes[7].values,
                                              #value = new_mon[3])
            #child_genes = child_genes.replace(child_genes[8].values,
                                              #value = new_mon[5])

# WHAT DO THESE FUNCTIONS DO?
                
#     elif index == 2:
#         new_gene, alternate = random.sample(all_genes[1], 2)
#         if child_genes[index].values == new_gene:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = alternate)
#         else:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = new_gene)
#     elif index == 3:
#         new_gene, alternate = random.sample(all_genes[2], 2)
#         if child_genes[index].values == new_gene:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = alternate)
#         else:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = alternate)
    
#     elif index == 9:
#         new_gene, alternate = random.sample(all_genes[4], 2)
#         if child_genes[index].values == new_gene:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = alternate)
#         else:
#             child_genes = child_genes.replace(child_genes[index].values,
#                                               value = alternate)
        
    child_genes = child_genes.rename(index = str, columns =
                                     {0: 'wt_IPDA', 1: 'wt_HDDA',
                                      2: 'wt_TMPTA', 3: 'wt_TMPEOTA',
                                      4: 'wt_DEM', 5: 'wt_DBM',
                                      6: 'wt_DOM', 7: 'wt_IPDI'})
    return child_genes


# This function switches one of the genes in the recipe dataframe with a
# new value and returns a dataframe with the new value to be passed to the
# fitness function to be evaluated again.


In [36]:
# Set up Fitness Function

def get_fitness(genes, target, prefer, limits):
    fitness = []
    pred = property_predictor(genes)

    if 'fisher_2h' in prefer:
        fitness.append(conditions[0](pred, target, limits[0]))
    if 'fisher_24h' in prefer:
        fitness.append(conditions[1](pred, target, limits[1]))
    if 'fisher_7d' in prefer:
        fitness.append(conditions[2](pred, target, limits[2]))
    if 'DOI_1d' in prefer:
        fitness.append(conditions[3](pred, target, limits[3]))
    if 'gloss_1d' in prefer:
        fitness.append(conditions[4](pred, target, limits[4]))
    if 'DOI_7d' in prefer:
        fitness.append(conditions[5](pred, target, limits[5]))
    if 'gloss_7d' in prefer:
        fitness.append(conditions[6](pred, target, limits[6]))
    if 'Amtec_afterScratch' in prefer:
        fitness.append(conditions[7](pred, target, limits[7]))
    if 'Amtec_afterReflow' in prefer:
        fitness.append(conditions[8](pred, target, limits[8]))
    if 'Amtect_afterScratchRel6600' in prefer:
        fitness.append(conditions[9](pred, target, limits[9]))
    if 'Amtec_afterReflowRel6600' in prefer:
        fitness.append(conditions[10](pred, target, limits[9]))
    return sum(fitness)

# this function determines the fitness of the recipe based on the users'
# defined limits or the default values

In [37]:
# Set up Iterative Functions

In [38]:
def get_best(get_fitness, optimal_fitness, gene_set,
             max_time, start_time, max_predictions, prefer, limits):
    random.seed()

    def fn_Mutate(parent):
        return _mutate(parent)

    def fn_Generate_Parent(gene_set):
        return _generate_parent(gene_set)

    for improvement in _get_improvement(fn_Mutate, fn_Generate_Parent,
                                        optimal_fitness, max_time, start_time,
                                        max_predictions, prefer, limits):
        if optimal_fitness < get_fitness(improvement, target,
                                         prefer, limits):
            return improvement

# This function continuously mutates and evaluates the recipe until its
# fitness is equal to or better than the optimal fitness.

In [39]:
def _get_improvement(mutate_parent, generate_parent,
                     optimal_fitness, max_time, start_time,
                     max_predictions, prefer, limits):
    elapsed_time = 0
    best_parent = _generate_parent(all_genes)
    num_predictions = 1

    yield best_parent
    while True:
        while elapsed_time < max_time and \
              num_predictions < max_predictions and \
              get_fitness(best_parent, target,
                          prefer, limits) < optimal_fitness:
            child = mutate_parent(best_parent)
            elapsed_time = time.time() - start_time
            predictions_list.append(property_predictor(child))
            recipes_list.append(child)
            fitness_list.append(get_fitness(child, target,
                                            prefer, limits))
            num_predictions = len(predictions_list)

            if get_fitness(best_parent, target, prefer, limits) > \
               get_fitness(child, target, prefer, limits):
                continue
            if not get_fitness(child, target, prefer, limits) > \
                   get_fitness(best_parent, target, prefer, limits):
                best_parent = child
                continue
            yield child
            best_parent = child
        break

# This function is part of the loop above, the added time and fitness
# constraints prevent the function from running indefinitely if no
# recipe can be found that matches the specified performance.

In [40]:
def determine_recipe(target, max_time, max_predictions, prefer, limits):
    gene_set = all_genes
    start_time = time.time()

    def fn_GetFitness(genes, target, prefer, limits):
        return get_fitness(genes, target, prefer, limits)

    if prefer != None:
        optimal_fitness = len(prefer)
    else:
        optimal_fitness = len(target.columns.values)

    get_best(fn_GetFitness, optimal_fitness, gene_set,
             max_time, start_time, max_predictions, prefer, limits)

    # This function puts all of the others together into one exectuable piece

In [41]:
# END OF GA FUNCTIONS

In [42]:
# DEFINE FUNCTIONS FOR GUI

def recipe_predict():
    global recipes_list
    global predictions_list
    global fitness_list
    global target
    global preferences
    global sort_pref
    global lh_sort_pref
    global pred_data
    global pred_limits

    recipes_list = []
    predictions_list = []
    fitness_list = []
    
    target = df(index = [2])
    target['fisher_2h'] = float(fisher2h.get())           # Check the naming convention here.
    target['fisher_24h'] = float(fisher24h.get()) 
    target['fisher_7d'] = float(fisher7d.get())
    target['DOI_1day'] = float(DOI1day.get())
    target['gloss_1day'] = float(gloss1day.get())
    target['DOI_7d'] = float(DOI7day.get())
    target['gloss_7day'] = float(gloss7day.get())
    target['Amtec_afterScratch'] = float(AmtecafterScratch.get())
    target['Amtec_afterReflow'] = float(AmtecafterReflow.get())
    target['Amtect_afterScratchRel6600'] = float(AmtectafterScratchRel6600.get())
    target['Amtec_afterReflowRel6600'] = float(AmtecafterReflowRel6600.get())

    preferences = [fisher2h_pref.get(), fisher24h_pref.get(),
                   fisher7d_pref.get(), DOI1day_pref.get(),
                   gloss1day_pref.get(), DOI7day_pref.get(),
                   gloss7day_pref.get(), AmtecafterScratch_pref.get(),
                   AmtecafterReflow_pref.get(), 
                   AmtectafterScratchRel6600_pref.get(), 
                   AmtecafterReflowRel6600_pref.get()]
    
    sort_pref = pref.get()
    lh_sort_pref = lh_pref.get()

    pred_limits = [[float(fisher2h_upper.get()),
                    float(fisher2h_lower.get())],
                   [float(fisher24h_upper.get()),
                    float(fisher24h_lower.get())],
                   [float(fisher7d_upper.get()),
                    float(fisher7d_lower.get())],
                   [float(DOI1day_upper.get()),
                    float(DOI1day_lower.get())],
                   [float(gloss1day_upper.get()),
                    float(gloss1day_lower.get())],
                   [float(DOI7day_upper.get()),
                    float(DOI7day_lower.get())],
                   [float(gloss7day_upper.get()),
                    float(gloss7day_lower.get())],
                   [float(AmtecafterScratch_upper.get()),
                    float(AmtecafterScratch_lower.get())],
                   [float(AmtecafterReflow_upper.get()),
                    float(AmtecafterReflow_lower.get())],
                   [float(AmtectafterScratchRel6600_upper.get()),
                    float(AmtectafterScratchRel6600_lower.get())],
                   [float(AmtecafterReflowRel6600_upper.get()),
                    float(AmtecafterReflowRel6600_lower.get())]]


    try:

        determine_recipe(target = target,
                         max_predictions = float(max_pred.get()),
                         max_time = float(max_time.get()),
                         prefer = [i for i in preferences if i != '0'],
                         limits = pred_limits)

        pred_data = df(np.vstack(recipes_list), columns =
                       ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
                        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'])

        new_columns = ['fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day',
                       'gloss_1day', 'DOI_7d', 'gloss_7d',
                       'Amtec_afterScratch', 'Amtec_afterReflow', 
                       'Amtect_afterScratchRel6600', 'Amtec_afterReflowRel6600']

        for properties, columns in zip(new_columns, np.arange(0,11,1)):
            pred_data[properties] = np.vstack(predictions_list)[:, columns]

#         thermal_columns = ['Theo Tg', 'Actual Tg', 'MFFT']

#         for temp in thermal_columns:
#             pred_data[temp] = pred_data[temp].values - 273.15

        pred_data['Fitness'] = np.vstack(fitness_list)
        pred_data = percent_scaler(pred_data).drop_duplicates()
        pred_data = pred_data[pred_data['Fitness'].values >= \
                              pred_data['Fitness'].max()]

        if lh_sort_pref == 'High to Low':
            pred_data = pred_data.sort_values(by = sort_pref,
                                              ascending = False
                                              ).reset_index(drop = 'True')
        else:
            pred_data = pred_data.sort_values(by = sort_pref
                                              ).reset_index(drop = 'True')

        return pred_data

    except:
        msgBox.showerror(title = 'PREDICTION ERROR!', message =
'''One or more of the target properties were not entered properly.''')

# this function determines recipes that best match the desired performance
# using the genetic algorithm defined above

In [43]:
def recipe_exporter(data, data_type, file_name):
    if '.xlsx' not in file_name:
        msgBox.showerror(title = 'EXPORT ERROR!', message =
'''File name does not contain .xlsx, please add this and re-export.''')

    else:
        if data_type == 'pred_data':
            workbook = xl_write.Workbook(file_name)
            worksheet1 = workbook.add_worksheet('Desired Properties')

            col = 0
            header = 0
            for name in target.columns.values:
                worksheet1.write(header, col, name)
                col = col + 1

            col = 0
            row = 1
            for value in target.iloc[0]:
                worksheet1.write(row, col, value)
                col = col + 1

            worksheet2 = workbook.add_worksheet('Predicted Recipes')

            col = 0
            header = 0
            for name in data.columns.values:
                worksheet2.write(header, col, name)
                col = col + 1

            for row in range(1, len(data)):
                col = 0
                for value in data.iloc[row]:
                    worksheet2.write(row, col, value)
                    col = col + 1

            workbook.close()

        if data_type == 'pred_results':
            workbook = xl_write.Workbook(file_name)
            worksheet1 = workbook.add_worksheet('Desired Properties')

            col = 0
            header = 0
            for name in target.columns.values:
                worksheet1.write(header, col, name)
                col = col + 1

            col = 0
            row = 1
            for value in target.iloc[0]:
                worksheet1.write(row, col, value)
                col = col + 1

            worksheet2 = workbook.add_worksheet('Predicted Recipe')

            col = 0
            row = 0
            for name in data[:,0]:
                worksheet2.write(row, col, name)
                col = col + 1

            col = 0
            row = 1
            for value in data[:,1]:
                worksheet2.write(row, col, value)
                col = col + 1
            workbook.close()

# This function allows the user to export prediction data into excel files

In [44]:
def make_recipe_prediction_window():
    recipe_predict()
    # this function envokes the genetic algorithm to determine a recipe to
    # match the users' specified performance attributes

    input_names = ['fisher_2h', 'fisher_24h',
                   'fisher_7d', 'DOI_1day',
                   'gloss_1day', 'DOI_7d',
                   'gloss_7d', 'Amtec_afterScratch',
                   'Amtec_afterReflow', 
                   'Amtect_afterScratchRel6600', 
                   'Amtec_afterReflowRel6600']
            
            
#     input_names = ['% Recovered', 'Actual Tg (C)',
#                    'MFFT (C)', 'Particle Size (nm)',
#                    '% Solids', 'Hardness (swings)',
#                    'Dry Adhesion', 'Wet Adhesion',
#                    'DPUR', 'HTPUR', 'PE Stability']

    errors = ['+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD',
              '+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD', '']

#     scale_labels = ['Adhesion Scale', 'DPUR Scale', 'HTPUR Scale']
#     scales = ['0-5', '0-5', '0-10']

    global inputs
    inputs = [k for i in target.transpose().values for k in i]
    inputs[1] = inputs[1] # - 273.15
    inputs[2] = inputs[2] # - 273.15

    pred_limits_df = df(inputs[:-1], columns = ['Values'])
    pred_limits_df['Upper Limits'] = [i[0] for i in pred_limits]
    pred_limits_df['Lower Limits'] = [i[1] for i in pred_limits]

    pred_limits_df = pred_limits_df.replace(0, value = '--')

#     if 'Actual Tg' in preferences and pred_limits_df.at[1,
#                                                         'Upper Limits'] != 0:
#         pred_limits_df.at[1, 'Values'] = '--'
#     else:
#         pred_limits_df.at[1, 'Values'] = float(act_tg.get())

#     if 'MFFT' in preferences and pred_limits_df.at[2,
#                                                    'Upper Limits'] != 0:
#         pred_limits_df.at[2, 'Values'] = '--'
#     else:
#         pred_limits_df.at[2, 'Values'] = float(mfft.get())

    upper_limits = pred_limits_df['Upper Limits'].values
    upper_limits[1] = upper_limits[1] # - 273.15
    upper_limits[2] = upper_limits[2] # - 273.15

    lower_limits = pred_limits_df['Lower Limits'].values
    lower_limits[1] = lower_limits[1] # - 273.15
    lower_limits[2] = lower_limits[2] # - 273.15

    if upper_limits[1] == 0 and lower_limits[1] == 0:
        upper_limits[1] = '--'
        lower_limits[1] = '--'

    if upper_limits[2] == 0 and lower_limits[2] == 0:
        upper_limits[2] = '--'
        lower_limits[2] = '--'

    fix_recipe_columns = ['wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
                        'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI']

    for col in fix_recipe_columns:
        if col == 'wt_IPDA' \
        or col == 'wt_HDDA' \
        or col == 'wt_TMPTA':
            pred_data[col] = [round(k, 2) for k in pred_data[col]]
        else:
            pred_data[col] = [round(k, 4) for k in pred_data[col]]
    # this loop rounds the values so that they don't display with several
    # decimal places that are irrelevant

    global pred_results

    pred_results = np.array([pred_data.columns.values,
                             pred_data.iloc[len(pred_data) - 1].values]
                            ).transpose()

    # the script below makes the recipe prediction window
    recipe_win = tk.Toplevel()
    recipe_win.title('PREDICTIONS')
    recipe_win.configure(bg = 'ghost white')

    for num in range(0,29): 
        for col in [0,10,21]:
            tk.Label(recipe_win, text = '', bg = 'ghost white').grid(
                     row = num, column = col)
    # this loop adds white borders to the window so that it's less busy

#####LOGO#####

    tk.Label(recipe_win, image = axalta_logo1).grid(
             row = 0, columnspan = 22)
    # this adds the Axalta logo

    tk.Label(recipe_win, text = 'DESIRED PROPERTIES', fg = 'dark blue',
             font = 'Times 13 bold italic underline',
             bg = 'ghost white').grid(
             row = 1, column = 1, columnspan = 9, sticky = W+E)

    tk.Label(recipe_win, text = 'Properties', font = 'Times 12 bold',
             relief = 'ridge', bg = 'ghost white', width = 15).grid(
             row = 2, column = 1, columnspan = 6, sticky = W+E)
    tk.Label(recipe_win, text = 'Values', font = 'Times 12 bold',
             relief = 'ridge', bg = 'ghost white').grid(
             row = 2, column = 7, sticky = W+E)
    for label, col in zip(['Upper Limit', 'Lower Limit'], [8,9]):
        tk.Label(recipe_win, text = label, font = 'Times 12 bold',
                 relief = 'ridge', bg = 'ghost white').grid(
                 row = 2, column = col, sticky = W+E)
        # this adds the column names
    for label, num in zip(input_names, range(3,14)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 1, columnspan = 6, sticky = W+E)
        # this adds the property names column

    tk.Label(recipe_win, text = 'Optimal Fitness', fg = 'Green',
             font = 'Times 12', bg = 'ghost white').grid(
             row = 14, column = 1, columnspan = 6, sticky = W+E)
    tk.Label(recipe_win, text = str(len([i for i in preferences
                                         if i != '0'])),
             font = 'Times 12', relief = 'ridge', bg = 'ghost white').grid(
             row = 14, column = 7, sticky = W+E)
    # this adds the optimal fitness value
    for label, num in zip(pred_limits_df['Values'].values, range(3,14)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 7, sticky = W+E)
    tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
             bg = 'ghost white', text = inputs[10]).grid(
             row = 13, column = 7, sticky = W+E)
        # this adds the input property values column
    for label, num in zip(upper_limits, range(3,14)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 8, sticky = W+E)
        # this adds the input property value upper limits column
    for label, num in zip(lower_limits, range(3,13)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 9, sticky = W+E)
        # this adds the input property value lower limits column
    # these loops add a table with the property specifications input by the
    # user

    tk.Label(recipe_win, text = 'PREDICTED PROPERTIES',
             fg = 'dark blue',
             font = 'Times 13 bold italic underline',
             bg = 'ghost white').grid(
             row = 1, column = 11, columnspan = 10, sticky = W+E)

    tk.Label(recipe_win, text = 'Properties', font = 'Times 12 bold',
             relief = 'ridge', bg = 'ghost white', width = 16).grid(
             row = 2, column = 11, columnspan = 8, sticky = W+E)

    for label, col in zip(['Values', 'Error'], [19,20]):
        tk.Label(recipe_win, text = label, font = 'Times 12 bold',
                 relief = 'ridge', bg = 'ghost white').grid(
                 row = 2, column = col, sticky = W+E)
        # this adds the column names for the predicted performance
    for label, row in zip(input_names, range(3,14)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = row, column = 11, columnspan = 8, sticky = W+E)
        # this adds the property names column

    tk.Label(recipe_win, text = 'Fitness Score', fg = 'Green',
             font = 'Times 12', bg = 'ghost white').grid(
             row = 14, column = 11, columnspan = 8, sticky = W+E)

    for label, row in zip(pred_results[10:23,1], range(3,15)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = row, column = 19, sticky = W+E)
        # this adds the property prediction results column from the 'best'
        # recipe determined by the genetic algorithm
    for label, row in zip(errors, range(3,15)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = row, column = 20, sticky = W+E)
        # this adds the error column for the property predictions
    # these loops add a table with the property predictions from the 'best'
    # recipe determined by the genetic algorithm

    for label, num in zip(scale_labels, range(17,20)):
        tk.Label(recipe_win, text = label, font = 'Times 12',
                 bg = 'ghost white', relief = 'ridge').grid(
                 row = num, column = 1, columnspan = 6, sticky = W+E)
    for label, num in zip(scales, range(17,20)):
        tk.Label(recipe_win, text = label, font = 'Times 12',
                 bg = 'ghost white', relief = 'ridge').grid(
                 row = num, column = 7, sticky = W+E)
    # these loops add a table with the scales of adhesion, DPUR and HTPUR

    tk.Label(recipe_win, text = \
'''Notes:
Higher values on the scales
are better.

All loadings are with respect to
toal monomer amount. (i.e. 1
equates to 1% of total monomer).''',
             bg = 'ghost white', font = 'Times 10', fg = 'red').grid(
             row = 20, rowspan = 7, column = 1, columnspan = 7, sticky = W+E)
    # this adds notes to be considered by the user


    tk.Label(recipe_win, text = 'POLYMER RECIPE', fg = 'dark green',
             font = 'Times 13 bold italic underline',
             bg = 'ghost white').grid(
             row = 16, column = 9, columnspan = 12, sticky = W+E)

    for label, value in zip(pred_results[0:10,0], range(17,27)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = value, column = 9, columnspan = 7, sticky = W+E)
        # this loop adds the recipe details column
    for label, value in zip(pred_results[0:10,1], range(17,27)):
        tk.Label(recipe_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = value, column = 16, columnspan = 5, sticky = W+E)
        # this loop adds the best recipe's values column
    # these loops add a table with the recipe that the genetic algorithm
    # determined to best match the users' specifications

    ttk.Style().configure('TButton')
    ttk.Button(recipe_win, text = 'EXPORT CURRENT RECIPE', command = \
               single_recipe_exporter).grid(
               row = 28, column = 1, columnspan = 20, sticky = W+E)
    # this button allows the user to export the recipe displayed in the window
    # into an excel file

    ttk.Button(recipe_win, text = 'EXPORT ALL RECIPES', command = \
               multi_recipe_exporter).grid(
               row = 29, column = 1, columnspan = 20, sticky = W+E)
    # this button allows the user to export all the recipes that have the
    # same fitness as the recipe in the window into an excel file

    ttk.Button(recipe_win, text = 'QUIT', command = recipe_win.destroy).grid(
               row = 30, column = 1, columnspan = 20, sticky = W+E)
    # this button allows the user to export all the recipes that have the
    # same fitness as the recipe in the window into an excel file


    recipe_win.mainloop()

    # this function makes the pop-up window showing the predicted recipe and
    # performance values determined by the genetic algorithm

In [45]:
def recipe_predictions():
    if msgBox.askyesno('Verify', message = 'Proceed with given properties?'):
        try:
            make_recipe_prediction_window()
        except:
            msgBox.showerror(title = 'PREDICTION ERROR!', message =
'''There was an error during predictions.''')

    else:
        msgBox.showinfo(message = 'Prediction has been cancelled.')

# this function makes a pop-up showing the predicted properties

In [46]:
def single_recipe_exporter():
    sr_export_win = tk.Tk()
    sr_export_win.title('Single Recipe Exporter')
    sr_export_win.configure(bg = 'ghost white')

    for i in [0,3]:
        tk.Label(sr_export_win, text = '', bg = 'ghost white').grid(
                 row = i, column = i)

    tk.Label(sr_export_win, font = 'Times 12', bg = 'ghost white', text =
'''Please enter the desired name of the excel file.
Be sure to add .xlsx to the end of the name.''').grid(
             row = 1, column = 1, columnspan = 2, sticky = W+E)

    tk.Label(sr_export_win, text = 'File Name', font = 'Times 12',
             bg = 'ghost white').grid(row = 2, column = 1)

    recipe_filename = tk.Entry(sr_export_win, font = 'Times 12',
                               textvariable = tk.StringVar(sr_export_win))
    recipe_filename.grid(row = 2, column = 2, sticky = W+E)

    ttk.Button(sr_export_win, text = 'EXPORT',
               command = lambda: recipe_exporter(data = pred_results,
                                                 data_type = 'pred_results',
                                                 file_name = \
                                                 recipe_filename.get())).grid(
               row = 3, column = 1, columnspan = 2, sticky = W+E)

    sr_export_win.mainloop()

# this function makes a window to export the predicted best recipe

In [47]:
def multi_recipe_exporter():
    mr_export_win = tk.Tk()
    mr_export_win.title('Multi Recipe Exporter')
    mr_export_win.configure(bg = 'ghost white')

    for i in [0,3]:
        tk.Label(mr_export_win, text = '', bg = 'ghost white').grid(
                 row = i, column = i)

    tk.Label(mr_export_win, font = 'Times 12', bg = 'gray92', text =
'''Please enter the desired name of the excel file.
Be sure to add .xlsx to the end of the name.''').grid(
             row = 1, column = 1, columnspan = 2, sticky = W+E)

    tk.Label(mr_export_win, text = 'File Name', font = 'Times 12',
             bg = 'ghost white').grid(row = 2, column = 1)

    recipes_filename = tk.Entry(mr_export_win, font = 'Times 12',
                                textvariable = tk.StringVar(mr_export_win))
    recipes_filename.grid(row = 2, column = 2, sticky = W+E)

    ttk.Button(mr_export_win, text = 'EXPORT',
               command = lambda: recipe_exporter(data = pred_data,
                                                 data_type = 'pred_data',
                                                 file_name = \
                                                 recipes_filename.get())).grid(
               row = 3, column = 1, columnspan = 2, sticky = W+E)

    mr_export_win.mainloop()

# this function makes a window to export the recipes that have the same
# fitness score as the best recipe

In [48]:
# CONSTRUCT GUI

def make_recipe_predictor():
    recipe_pred = tk.Toplevel()
    recipe_pred.title(' POLYMER RECIPE PREDICTOR ')
    recipe_pred.configure(background = 'gray92') 
    
    # set all global variables
    global fisher2h, fisher2h_pref, fisher2h_upper, fisher2h_lower
    global fisher24h, fisher24h_pref, fisher24h_upper, fisher24h_lower
    global fisher7d, fisher7d_pref, fisher7d_upper, fisher7d_lower
    global DOI1day, DOI1day_pref, DOI1day_upper, DOI1day_lower
    global gloss1day, gloss1day_pref, gloss1day_upper, gloss1day_lower
    global DOI7day, DOI7day_pref, DOI7day_upper, DOI7day_lower
    global gloss7day, gloss7day_pref, gloss7day_upper, gloss7day_lower
    global AmtecafterScratch, AmtecafterScratch_pref, AmtecafterScratch_upper, AmtecafterScratch_lower
    global AmtecafterReflow, AmtecafterReflow_pref, AmtecafterReflow_upper, AmtecafterReflow_lower
    global AmtectafterScratchRel6600, AmtectafterScratchRel6600_pref, AmtectafterScratchRel6600_upper, AmtectafterScratchRel6600_lower
    global AmtecafterReflowRel6600, AmtecafterReflowRel6600_pref, AmtecafterReflowRel6600_upper, AmtecafterReflowRel6600_lower
#     global htpur, htpur_pref, htpur_upper, htpur_lower
#     global pe_stab, pe_stab_pref
#     global max_pred, max_time, pref, lh_pref
    global axalta_logo1


    for num in range(0,25):
        tk.Label(recipe_pred, text = '', bg = 'gray92').grid(
                 row = num, column = 0)
        # This loop adds white borders to the window so that it does not look so busy.

# #####LOGO#####
        
#     axalta_logo1 = ImageTk.PhotoImage(Image.open('axaltalogo.png'))
#     tk.Label(recipe_pred, image = axalta_logo1).grid(
#              row = 0, columnspan = 7)
#     # this adds the Axalta logo to the window

    tk.Label(recipe_pred, text = 
'''Please enter the desired performance values below and check
the boxes for the properties that are to be optimized.''',
             bg = 'gray92', font = 'Times 13', fg = 'green').grid(
             row = 2, column = 0, columnspan = 7, sticky = W+E)
    # this adds a note to be considered by the user

    for names, cols in zip(['Values', 'Upper Limit', 'Lower Limit'], [2,3,4]):
        tk.Label(recipe_pred, text = names, relief = 'ridge',
                 bg = 'gray92', font = 'Times 13 bold').grid(
                 row = 4, column = cols, sticky = W+E)
    # this loop adds column headers for the entry values



    # Add Entry Fields for Specified Performance
    
# 'fisher_2h', 'fisher_24h', 'fisher_7d', 'DOI_1day', 'gloss_1day', 'DOI_7d',
# 'gloss_7d', 'Amtec_afterScratch', 'Amtec_afterReflow', 'Amtect_afterScratchRel6600', 
# 'Amtec_afterReflowRel6600'

        # Fisher 2h hardness
    tk.Label(recipe_pred, text = 'Fisher 2h hardness',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 5, sticky = W, padx = 30)
    fisher2h = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    fisher2h.grid(column = 2, row = 5, sticky = W+E)
        # this adds the Fisher2h entry field

    fisher2h_pref = tk.StringVar(recipe_pred)
    fisher2h_pref.set('fisher_2h')
    button1 = tk.Checkbutton(recipe_pred, variable = fisher2h_pref,
                             onvalue = 'fisher_2h',
                             background = 'gray92')
    button1.grid(column = 1, row = 5, sticky = W)
        # this adds a preference box for Fisher2h

    fisher2h_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher2h_upper.grid(column = 3, row = 5, sticky = W+E)
        # this adds the Fisher2h upper limit entry field
    fisher2h_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher2h_lower.grid(column = 4, row = 5, sticky = W+E)
        # this adds the Fisher2h lower limit entry field

# Next, all rows change from row = 5 to row = 6

        # Fisher 24h hardness
    tk.Label(recipe_pred, text = 'Fisher 24h hardness',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 6, sticky = W, padx = 30)
    fisher24h = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    fisher24h.grid(column = 2, row = 6, sticky = W+E)
        # this adds the Fisher24h entry field

    fisher24h_pref = tk.StringVar(recipe_pred)
    fisher24h_pref.set('fisher_24h')
    button2 = tk.Checkbutton(recipe_pred, variable = fisher2h_pref,
                             onvalue = 'fisher_24h',
                             background = 'gray92')
    button2.grid(column = 1, row = 6, sticky = W)
        # this adds a preference box for Fisher24h

    fisher24h_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher24h_upper.grid(column = 3, row = 6, sticky = W+E)
        # this adds the Fisher24h upper limit entry field
    fisher24h_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher24h_lower.grid(column = 4, row = 6, sticky = W+E)
        # this adds the Fisher24h lower limit entry field

# Next, all rows change from row = 6 to row = 7 (and step up button number to 3)

        # Fisher 7d hardness
    tk.Label(recipe_pred, text = 'Fisher 7d hardness',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 7, sticky = W, padx = 30)
    fisher7d = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    fisher7d.grid(column = 2, row = 7, sticky = W+E)
        # this adds the Fisher7d entry field

    fisher7d_pref = tk.StringVar(recipe_pred)
    fisher7d_pref.set('fisher_7d')
    button3 = tk.Checkbutton(recipe_pred, variable = fisher7d_pref,
                             onvalue = 'fisher_7d',
                             background = 'gray92')
    button3.grid(column = 1, row = 7, sticky = W)
        # this adds a preference box for Fisher7d

    fisher7d_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher7d_upper.grid(column = 3, row = 7, sticky = W+E)
        # this adds the Fisher7d upper limit entry field
    fisher7d_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    fisher7d_lower.grid(column = 4, row = 7, sticky = W+E)
        # this adds the Fisher7d lower limit entry field
        
# Next, all rows change from row = 7 to row = 8 (and step up button number to 4)

        # DOI 1day
    tk.Label(recipe_pred, text = 'DOI 1day',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 8, sticky = W, padx = 30)
    DOI1day = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    DOI1day.grid(column = 2, row = 8, sticky = W+E)
        # this adds the DOI 1day entry field

    DOI1day_pref = tk.StringVar(recipe_pred)
    DOI1day_pref.set('DOI_1day')
    button4 = tk.Checkbutton(recipe_pred, variable = DOI1day_pref,
                             onvalue = 'DOI_1day',
                             background = 'gray92')
    button4.grid(column = 1, row = 8, sticky = W)
        # this adds a preference box for DOI 1day

    DOI1day_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    DOI1day_upper.grid(column = 3, row = 8, sticky = W+E)
        # this adds the DOI 1day upper limit entry field
    DOI1day_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    DOI1day_lower.grid(column = 4, row = 8, sticky = W+E)
        # this adds the DOI 1day lower limit entry field
        
# Next, all rows change from row = 8 to row = 9 (and step up button number to 5)

        # Gloss 1day
    tk.Label(recipe_pred, text = 'Gloss 1day',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 9, sticky = W, padx = 30)
    gloss1day = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    gloss1day.grid(column = 2, row = 9, sticky = W+E)
        # this adds the gloss 1day entry field

    gloss1day_pref = tk.StringVar(recipe_pred)
    gloss1day_pref.set('gloss_1day')
    button5 = tk.Checkbutton(recipe_pred, variable = gloss1day_pref,
                             onvalue = 'gloss_1day',
                             background = 'gray92')
    button5.grid(column = 1, row = 9, sticky = W)
        # this adds a preference box for gloss 1day

    gloss1day_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    gloss1day_upper.grid(column = 3, row = 9, sticky = W+E)
        # this adds the gloss 1day upper limit entry field
    gloss1day_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    gloss1day_lower.grid(column = 4, row = 9, sticky = W+E)
        # this adds the gloss 1day lower limit entry field      
        
 # Next, all rows change from row = 9 to row = 10 (and step up button number to 6)

        # DOI 7day
    tk.Label(recipe_pred, text = 'DOI 7day',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 10, sticky = W, padx = 30)
    DOI7day = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    DOI7day.grid(column = 2, row = 10, sticky = W+E)
        # this adds the DOI 7day entry field

    DOI7day_pref = tk.StringVar(recipe_pred)
    DOI7day_pref.set('DOI_7day')
    button6 = tk.Checkbutton(recipe_pred, variable = DOI1day_pref,
                             onvalue = 'DOI_7day',
                             background = 'gray92')
    button6.grid(column = 1, row = 10, sticky = W)
        # this adds a preference box for DOI 7day

    DOI7day_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    DOI7day_upper.grid(column = 3, row = 10, sticky = W+E)
        # this adds the DOI 7day upper limit entry field
    DOI7day_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    DOI7day_lower.grid(column = 4, row = 10, sticky = W+E)
        # this adds the DOI 1day lower limit entry field       

# Next, all rows change from row = 10 to row = 11 (and step up button number to 7)

        # Gloss 7day
    tk.Label(recipe_pred, text = 'Gloss 7day',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 11, sticky = W, padx = 30)
    gloss7day = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    gloss7day.grid(column = 2, row = 11, sticky = W+E)
        # this adds the gloss 7day entry field

    gloss7day_pref = tk.StringVar(recipe_pred)
    gloss7day_pref.set('gloss_7day')
    button7 = tk.Checkbutton(recipe_pred, variable = gloss7day_pref,
                             onvalue = 'gloss_7day',
                             background = 'gray92')
    button7.grid(column = 1, row = 11, sticky = W)
        # this adds a preference box for gloss 7day

    gloss7day_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    gloss7day_upper.grid(column = 3, row = 11, sticky = W+E)
        # this adds the gloss 7day upper limit entry field
    gloss7day_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    gloss7day_lower.grid(column = 4, row = 11, sticky = W+E)
        # this adds the gloss 7day lower limit entry field  


# Next, all rows change from row = 11 to row = 12 (and step up button number to 8)

        # Amtec_afterScratch
    tk.Label(recipe_pred, text = 'Amtec after scratch',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 12, sticky = W, padx = 30)
    AmtecafterScratch = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    AmtecafterScratch.grid(column = 2, row = 12, sticky = W+E)
        # this adds the Amtec_afterScratch entry field

    AmtecafterScratch_pref = tk.StringVar(recipe_pred)
    AmtecafterScratch_pref.set('Amtec_afterScratch')
    button8 = tk.Checkbutton(recipe_pred, variable = AmtecafterScratch_pref,
                             onvalue = 'Amtec_afterScratch',
                             background = 'gray92')
    button8.grid(column = 1, row = 12, sticky = W)
        # this adds a preference box for Amtec_afterScratch

    AmtecafterScratch_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterScratch_upper.grid(column = 3, row = 12, sticky = W+E)
        # this adds the Amtec_afterScratch upper limit entry field
    AmtecafterScratch_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterScratch_lower.grid(column = 4, row = 12, sticky = W+E)
        # this adds the Amtec_afterScratch lower limit entry field  


# Next, all rows change from row = 12 to row = 13 (and step up button number to 9)

        # Amtec_afterReflow
    tk.Label(recipe_pred, text = 'Amtec after reflow',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 13, sticky = W, padx = 30)
    AmtecafterReflow = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    AmtecafterReflow.grid(column = 2, row = 13, sticky = W+E)
        # this adds the Amtec_afterReflow entry field

    AmtecafterReflow_pref = tk.StringVar(recipe_pred)
    AmtecafterReflow_pref.set('Amtec_afterReflow')
    button9 = tk.Checkbutton(recipe_pred, variable = AmtecafterReflow_pref,
                             onvalue = 'Amtec_afterReflow',
                             background = 'gray92')
    button9.grid(column = 1, row = 13, sticky = W)
        # this adds a preference box for Amtec_afterReflow

    AmtecafterReflow_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterReflow_upper.grid(column = 3, row = 13, sticky = W+E)
        # this adds the Amtec_afterReflow upper limit entry field
    AmtecafterReflow_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterReflow_lower.grid(column = 4, row = 13, sticky = W+E)
        # this adds the Amtec_afterReflow lower limit entry field  


# Next, all rows change from row = 13 to row = 14 (and step up button number to 10)

        # Amtect_afterScratchRel6600
    tk.Label(recipe_pred, text = 'Amtec after scratch rel 6600',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 14, sticky = W, padx = 30)
    AmtectafterScratchRel6600 = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    AmtectafterScratchRel6600.grid(column = 2, row = 14, sticky = W+E)
        # this adds the Amtect_afterScratchRel6600 entry field

    AmtectafterScratchRel6600_pref = tk.StringVar(recipe_pred)
    AmtectafterScratchRel6600_pref.set('Amtect_afterScratchRel6600')
    button10 = tk.Checkbutton(recipe_pred, variable = AmtecafterReflow_pref,
                             onvalue = 'Amtect_afterScratchRel6600',
                             background = 'gray92')
    button10.grid(column = 1, row = 14, sticky = W)
        # this adds a preference box for Amtect_afterScratchRel6600

    AmtectafterScratchRel6600_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtectafterScratchRel6600_upper.grid(column = 3, row = 14, sticky = W+E)
        # this adds the Amtect_afterScratchRel6600 upper limit entry field
    AmtectafterScratchRel6600_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtectafterScratchRel6600_lower.grid(column = 4, row = 14, sticky = W+E)
        # this adds the Amtect_afterScratchRel6600 lower limit entry field 


# Next, all rows change from row = 14 to row = 15 (and step up button number to 11)

        # Amtec_afterReflowRel6600
    tk.Label(recipe_pred, text = 'Amtec after reflow rel 6600',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 15, sticky = W, padx = 30)
    AmtecafterReflowRel6600 = tk.Entry(recipe_pred,
                         textvariable = tk.DoubleVar(recipe_pred),
                         width = 8, font = 'Arial 12')
    AmtecafterReflowRel6600.grid(column = 2, row = 15, sticky = W+E)
        # this adds the Amtec_afterReflowRel6600 entry field

    AmtecafterReflowRel6600_pref = tk.StringVar(recipe_pred)
    AmtecafterReflowRel6600_pref.set('Amtec_afterReflowRel6600')
    button11 = tk.Checkbutton(recipe_pred, variable = AmtecafterReflowRel6600_pref,
                             onvalue = 'Amtec_afterReflowRel6600',
                             background = 'gray92')
    button11.grid(column = 1, row = 15, sticky = W)
        # this adds a preference box for Amtec_afterReflowRel6600

    AmtecafterReflowRel6600_upper = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterReflowRel6600_upper.grid(column = 3, row = 15, sticky = W+E)
        # this adds the Amtec_afterReflowRel6600 upper limit entry field
    AmtecafterReflowRel6600_lower = tk.Entry(recipe_pred,
                               textvariable = tk.DoubleVar(recipe_pred),
                               width = 8, font = 'Arial 12')
    AmtecafterReflowRel6600_lower.grid(column = 4, row = 15, sticky = W+E)
        # this adds the Amtec_afterReflowRel6600 lower limit entry field 


    tk.Label(recipe_pred, bg = 'gray92', fg = 'red', font = 'Times 11',
             justify = 'left', text =
'''Not all values need to be entered; however, values should be entered for \
all properties that are checked.''').grid(
             column = 0, columnspan = 9, row = 16, sticky = W+E)

    tk.Label(recipe_pred, background = 'gray92',
             text = 'Please enter the genetic algorithm parameters below.',
             font = 'Times 13', fg = 'green').grid(
             column = 1, columnspan = 7, row = 18, sticky = W+E)
    # these add notes to be considered by the user


    # Add GA specific entries
    tk.Label(recipe_pred, text = 'Maximum Number of Predictions',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, columnspan = 2, row = 19, sticky = W)
    max_pred = tk.Entry(recipe_pred,
                        textvariable = tk.DoubleVar(recipe_pred),
                        width = 8, font = 'Arial 12')
    max_pred.grid(column = 3, columnspan = 2, row = 19, sticky = W+E)
        # this adds the maximum prediction entry field

    tk.Label(recipe_pred, text = 'Time Limit for Predictions (sec)',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, columnspan = 2, row = 20, sticky = W)
    max_time = tk.Entry(recipe_pred,
                        textvariable = tk.DoubleVar(recipe_pred),
                        width = 8, font = 'Arial 12')
    max_time.grid(column = 3, columnspan = 2, row = 20, sticky = W+E)
        # this adds the maximum prediction entry field


    # Add Drop-Down Menus for Sorting Preference Selection
    ttk.Style().configure('TMenubutton', background = 'ghost white',
                          font = 'Times 13')
    ttk.Label(recipe_pred, text = 'Highest Property Preference',
              background = 'ghost white',
              font = 'Times 13').grid(column = 1, row = 21, sticky = W)

    pref = tk.StringVar(recipe_pred)
    all_pref = ['Select', 'fisher_2h', 'fisher_24h', 'fisher_7d',
                'DOI_1day', 'gloss_1day', 'DOI_7d', 'gloss_7d',
                'Amtec_afterScratch', 'Amtec_afterReflow', 'Amtect_afterScratchRel660',
               'Amtec_afterReflowRel6600']
    drop1 = ttk.OptionMenu(recipe_pred, pref, *all_pref)
    drop1.grid(column = 3, columnspan = 2, row = 21, sticky = W+E)
        # this adds the first sorting preference drop-down menu


    ttk.Style().configure('TMenubutton', background = 'ghost white',
                          font = 'Times 13')
    ttk.Label(recipe_pred, text = 'Sorting Preference',
              background = 'ghost white',
              font = 'Times 13').grid(column = 1, row = 22, sticky = W)

    lh_pref = tk.StringVar(recipe_pred)
    all_lh_pref = ['Select', 'Low to High', 'High to Low']
    drop1 = ttk.OptionMenu(recipe_pred, lh_pref, *all_lh_pref)
    drop1.grid(column = 3, columnspan = 2, row = 22, sticky = W+E)
        # this adds the second sorting preference drop-down menu


    # Add Buttons
    ttk.Style().configure('TButton')

    ttk.Button(recipe_pred, text = 'DETERMINE RECIPE',
               command = recipe_predictions).grid(
               column = 1, columnspan = 4, row = 24, sticky = W+E)
        # this adds the button to display polymer property predictions

    ttk.Button(recipe_pred, text = 'QUIT',
               command = recipe_pred.destroy).grid(
               column = 1, columnspan = 4, row = 25, sticky = W+E)
        # this adds the button to display polymer property predictions


    recipe_pred.mainloop()

In [49]:
# END OF RECIPE PREDICTING GUI CODE

In [50]:
# PROPERTY PREDICTING GUI CODE

In [51]:
# Define needed functions

def property_exporter(data, file_name):
    if 'xlsx' not in file_name:
        msgBox.showerror(title = 'EXPORT ERROR!', message =
'''File name does not contain .xlsx, please add this and re-export.''')

    else:
        workbook = xl_write.Workbook(file_name)

        # Add Worksheet 1 for the Polymer Recipe
        worksheet1 = workbook.add_worksheet('Polymer Recipe')

        col = 0
        header = 0
        for name in user_input.columns.values:
            worksheet1.write(header, col, name)
            col = col + 1

        col = 0
        row = 1
        for value in user_input.iloc[0]:
            worksheet1.write(row, col, value)
            col = col + 1

        # Add Worksheet 2 for the Predicted Properties of the Polymer
        worksheet2 = workbook.add_worksheet('Predicted Properties')

        col = 0
        header = 0

        for name in prop_predictions.index:
            worksheet2.write(header, col, name)
            col = col + 1

        col = 0
        row = 1
        for value in prop_predictions[0]:
            worksheet2.write(row, col, value)
            col = col + 1

        row = 2
        col = 0
        for value in prop_predictions[1]:
            worksheet2.write(row, col, value)
            col = col + 1

        workbook.close()

# This overall function allows the user to export data into an excel file

In [52]:
# 'wt_IPDA', 'wt_HDDA', 'wt_TMPTA', 'wt_TMPEOTA',
# 'wt_DEM', 'wt_DBM', 'wt_DOM', 'wt_IPDI'

In [53]:
def show_monomer():
    try:
        msgBox.showinfo(message = 'Total monomer loading = %.4f' %
                        (sum([float(wt_IPDA_load.get()),
                              float(wt_HDDA_load.get()),
                              float(wt_TMPTA_load.get()),
                              float(wt_TMPEOTA_load.get()),
                              float(wt_DEM_load.get()),
                              float(wt_DBM_load.get()),
                              float(wt_DOM_load.get()),
                              float(wt_IPDI_load.get()),
                             ])))
    except:
        msgBox.showerror(title = 'RECIPE ERROR!', message =
'''One or more of the recipe components were not entered properly.''')

# this oeverall function makes a pop-up showing the total monomer amount

In [54]:
def predictions():
    if msgBox.askyesno('Verify', message = 'Proceed with given recipe?'):
        try:
            if round(sum([float(wt_IPDA_load.get()),
                              float(wt_HDDA_load.get()),
                              float(wt_TMPTA_load.get()),
                              float(wt_TMPEOTA_load.get()),
                              float(wt_DEM_load.get()),
                              float(wt_DBM_load.get()),
                              float(wt_DOM_load.get()),
                              float(wt_IPDI_load.get()),
                         ]), 4) < 100.0:
                msgBox.showerror(title = 'RECIPE ERROR!', message =
                            '''Total monomer loading is less than 100%.''')
            # this exception is raised if total monomer loading is less than
            # 100, in which case predictions will not proceed until corrected

            elif round(sum([float(wt_IPDA_load.get()),
                              float(wt_HDDA_load.get()),
                              float(wt_TMPTA_load.get()),
                              float(wt_TMPEOTA_load.get()),
                              float(wt_DEM_load.get()),
                              float(wt_DBM_load.get()),
                              float(wt_DOM_load.get()),
                              float(wt_IPDI_load.get()),
                           ]), 4) > 100.0:
                msgBox.showerror(title = 'RECIPE ERROR!', message =
                                 '''Total monomer loading exceeds 100%.''')
            # this exception is raised if total monomer loading is greater than
            # 100, in which case predictions will not proceed until corrected

#             elif surf_load.get() == '0' or surf_load.get() == '0.0':
#                 msgBox.showerror(title = 'RECIPE ERROR!', message =
#                                  '''No surfactant added.''')
#             # this exception is raised if no surfactant has been added
#             # predictions will not proceed until corrected

#             elif surf.get() == 'PAM 200':
#                 msgBox.showerror(title = 'RECIPE ERROR!', message =
#                                  '''No data for PAM 200 yet.''')
#             # this exception is raised if PAM 200 is selected as a surfactant
#             # since no batches have been made with this surfactant, the
#             # predictions would be erroneous, it's currently here because it
#             # will be added in the future

#             elif float(sty_load.get()) > 0:
#                 msgBox.showerror(title = 'RECIPE ERROR!', message =
#                                  '''No data for styrene yet.''')
#             # this exception is raised for the same reason as PAM 200, the
#             # data will be added in the future

######CHANGE HERE!!!!!###########

#             else:
#                 try:
#                     make_property_prediction_window()
#                 except:
#                     msgBox.showerror(title = 'RECIPE ERROR!', message =
# '''One or more of the recipe components were not entered properly.''')
#             # this exception is raised when the user either inputs a string in
#             # one of the entry boxes or did not select a surfactant or
#             # promoter from the drop-down menus

        except:
            msgBox.showerror(title = 'RECIPE ERROR!', message =
'''One or more of the recipe components were not entered properly.''')
            # this exception is raised if something else went wrong in the
            # predictions that was not caught by the previous exceptions

    else:
        msgBox.showinfo(message = 'Prediction has been cancelled.')

# this overall function makes a pop-up showing the predicted properties

In [55]:
def exporter():
    prop_export_win = tk.Tk()
    prop_export_win.title('Property Exporter')
    prop_export_win.configure(bg = 'gray92')

    for i in [0,3]:
        tk.Label(prop_export_win, text = '', bg = 'gray92').grid(
                 row = i, column = i)

    tk.Label(prop_export_win, font = 'Times 12', bg = 'ghost white', text =
'''Please enter the desired name of the excel file.
Be sure to add .xlsx to the end of the name.''').grid(
             row = 1, column = 1, columnspan = 2, sticky = W+E)

    tk.Label(prop_export_win, text = 'File Name', font = 'Times 12',
             bg = 'ghost white').grid(row = 2, column = 1)

    property_filename = tk.Entry(prop_export_win, font = 'Times 12',
                                 textvariable = tk.StringVar(prop_export_win))
    property_filename.grid(row = 2, column = 2, sticky = W+E)

    ttk.Button(prop_export_win, text = 'EXPORT',
               command = lambda: property_exporter(user_input,
                                                   property_filename.get())
                                                   ).grid(
               row = 3, column = 1, columnspan = 2, sticky = W+E)

    prop_export_win.mainloop()

# this function makes a window to export the property prediction results

In [56]:

def make_property_prediction_window():
    global user_input
    global prop_predictions

    user_input = df(index = [2])
    user_input['wt_IPDA'] = float(wt_IPDA_load.get())/100
    user_input['wt_HDDA'] = float(wt_HDDA_load.get())/100
    user_input['wt_TMPTA'] = float(wt_TMPTA_load.get())/100
    user_input['wt_TMPEOTA'] = float(wt_TMPEOTA_load.get())/100
    user_input['wt_DEM'] = float(wt_DEM_load.get())/100
    user_input['wt_DBM'] = float(wt_DBM_load.get())/100
    user_input['wt_DOM'] = float(wt_DOM_load.get())/100
    user_input['wt_IPDI'] = float(wt_IPDI_load.get())/100
   

    prop_predictions = property_predictor(user_input)

#     fix_temps = ['Actual Tg', 'MFFT']

#     for temp in fix_temps:
#         prop_predictions[temp] = (prop_predictions[temp].values - 
#                                   273.15).round(2)

    error = df(np.matrix(['+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD',
                          '+/-  TBD', '+/-  TBD', '+/-  TBD', '+/-  TBD',
                          '+/-  TBD', '+/-  TBD', '']), index = [1],
               columns = ['fisher_2h', 'fisher_24h',
                   'fisher_7d', 'DOI_1day',
                   'gloss_1day', 'DOI_7d',
                   'gloss_7d', 'Amtec_afterScratch',
                   'Amtec_afterReflow', 
                   'Amtect_afterScratchRel6600', 
                   'Amtec_afterReflowRel6600'])

    prop_predictions = prop_predictions.append(error).transpose()

    input_names = ['wt_IPDA', 'wt_HDDA', 
                   'wt_TMPTA','wt_TMPEOTA', 
                   'wt_DEM','wt_DBM', 
                   'wt_DOM','wt_IPDI']

    inputs = [
        float(wt_IPDA_load.get()),
        float(wt_HDDA_load.get()),
        float(wt_TMPTA_load.get()),
        float(wt_TMPEOTA_load.get()),
        float(wt_DEM_load.get()),
        float(wt_DBM_load.get()),
        float(wt_DOM_load.get()),
        float(wt_IPDI_load.get())
    ]

#     scale_labels = ['Adhesion Scale', 'DPUR Scale', 'HTPUR Scale']
#     scales = [' 0-5 ', ' 0-5 ', ' 0-10 ']

    prop_win = tk.Toplevel()
    prop_win.title('PREDICTIONS')
    prop_win.configure(bg = 'grey92')

    axalta_logo = ImageTk.PhotoImage(Image.open('axaltalogo.png'))
    tk.Label(prop_win, image = axalta_logo).grid(
             row = 0, columnspan = 8)
    # this adds the behr logo to the top of the window

    tk.Label(prop_win, text = 'POLYMER RECIPE', fg = 'dark green',
             font = 'Times 14 bold italic underline',
             bg = 'ghost white').grid(
             row = 1, columnspan = 6, sticky = W, padx = 100)
    for label, num in zip(input_names, range(2,11)):
        tk.Label(prop_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 1, columnspan = 2, sticky = W+E)
    for label, num in zip(inputs, range(2,11)):
        tk.Label(prop_win, font = 'Times 12', relief = 'ridge',
                 bg = 'ghost white', text = label).grid(
                 row = num, column = 3, columnspan = 2, sticky = W+E)
    # these loops add the two columns of the input table

    for num in range(1,10):
        for col in [0,7]:
            tk.Label(prop_win, text = '', bg = 'ghost white').grid(
                     row = num, column = col)
    for num in range(11,28):
        for col in [0,4,7]:
            tk.Label(prop_win, text = '', bg = 'ghost white').grid(
                     row = num, column = col)
    # these loops add borders of white to the window so that its less busy

    beaker = ImageTk.PhotoImage(Image.open('beaker.jpg'))
    tk.Label(prop_win, image = beaker).grid(
             row = 2, rowspan = 9, column = 5, columnspan = 3)
    # this adds a custom beaker image to the window

    tk.Label(prop_win, text = 'PREDICTED PROPERTIES', fg = 'dark blue',
             font = 'Times 14 bold italic underline',
             bg = 'ghost white').grid(
             row = 12, columnspan = 7, sticky = W, padx = 60)

    for label, num in zip(['Property', 'Prediction', 'Error'], range(1,4)):
        tk.Label(prop_win, text = label, relief = 'ridge',
                 font = 'Times 14 bold', bg = 'ghost white').grid(
                 row = 13, column = num, sticky = W+E)
    for label, num in zip(list(prop_predictions.index), range(14,25)):
        tk.Label(prop_win, bg = 'ghost white', font = 'Times 12',
                 borderwidth = 2, relief = 'ridge', text = label).grid(
                 row = num, column = 1, sticky = W+E)
    for label, num in zip(list(prop_predictions[0]), range(14,25)):
        tk.Label(prop_win, bg = 'ghost white', font = 'Times 12',
                 borderwidth = 2, relief = 'ridge',text = label).grid(
                 row = num, column = 2, sticky = W+E)
    for label, num in zip(list(prop_predictions[1]), range(14,25)):
        tk.Label(prop_win, bg = 'ghost white', font = 'Times 12',
                 borderwidth = 2, relief = 'ridge',text = label).grid(
                 row = num, column = 3, sticky = W+E)
    # these loops add the four columns of the predicted properties table

    for label, num in zip(scale_labels, range(14,17)):
        tk.Label(prop_win, text = label, font = 'Times 12',
                 bg = 'ghost white', relief = 'ridge').grid(
                 row = num, column = 5, sticky = W+E)
    for label, num in zip(scales, range(14,17)):
        tk.Label(prop_win, text = label, font = 'Times 12',
                 bg = 'ghost white', relief = 'ridge').grid(
                 row = num, column = 6, sticky = W+E)
    # these loops add a table with the scales of adhesion, DPUR and HTPUR

#     tk.Label(prop_win, text = \
# '''Notes: Adhesion here
# is adhesion to
# unprepared concrete.

# Higher values on the
# scales are better.''',
#              bg = 'ghost white', font = 'Times 10', fg = 'red').grid(
#              row = 17, rowspan = 7, column = 5, columnspan = 2, sticky = W+E)
#     # this adds notes to be considered by the user

    ttk.Button(prop_win, text = 'EXPORT RESULTS', command = exporter).grid(
               row = 26, column = 1, columnspan = 6, sticky = W+E)
    # this adds a button to export the current property predictions and
    # recipe to an excel file

    ttk.Button(prop_win, text = 'EXIT', command = prop_win.destroy).grid(
               row = 27, column = 1, columnspan = 6, sticky = W+E)


    prop_win.mainloop()

# this overall function makes a pop-up window with the polymer's recipe 
# and predicted properties

In [57]:
# CONSTRUCT GUI

def make_property_predictor():
    prop_pred = tk.Toplevel()
    prop_pred.title(' TOPCOAT PROPERTY PREDICTOR ')
    prop_pred.configure(background = 'gray92')
    
    # set all variables to global
    global wt_IPDA_load, wt_HDDA_load, wt_TMPTA_load, wt_TMPEOTA_load
    global wt_DEM_load, wt_DBM_load, wt_DOM_load, wt_IPDI_load   
    
    # Add Borders
    for num in range(0,14):
        for col in [0,2]:
            tk.Label(prop_pred, text = '', bg = 'gray92').grid(
                     row = num, column = col)

    # Add Logo
    axalta = Image.open('axaltalogo.png')
    #axalta_small = axalta.resize((64, 64), axalta.ANTIALIAS)
    axalta_logo = ImageTk.PhotoImage(axalta)
    tk.Label(prop_pred, image = axalta_logo).grid(
             row = 0, columnspan = 3)

#     # Add Drop-Down Menus for Material Selection
#     ttk.Style().configure('TMenubutton', background = 'ghost white',
#                           font = 'Times 13')

#     ttk.Label(prop_pred, text = 'Adhesion Promoter',
#               background = 'ghost white',
#               font = 'Times 13').grid(column = 1, row = 1, sticky = W)
#     prom = tk.StringVar(prop_pred)
#     drop1 = ttk.OptionMenu(prop_pred, prom,
#                            *['Select', 'And 174', 'PAM 4000', 'WAM II'])
#     drop1.grid(column = 1, row = 1, sticky = E)
#         # this adds the adhesion promoter drop-down menu

#     tk.Label(prop_pred, text = 'Surfactant', background = 'ghost white',
#              font = 'Times 13').grid(column = 1, row = 2, sticky = W)
#     surf = tk.StringVar(prop_pred)
#     drop2 = ttk.OptionMenu(prop_pred, surf, 
#                            *['Select', 'ER-10', 'PAM 100',
#                              'PAM 200', 'SR-1025'])
#     drop2.grid(column = 1, row = 2, sticky = E)
#     # this adds the surfactant drop-down menu


#     # set all variables to global
#     global wt_IPDA_load, wt_HDDA_load, wt_TMPTA_load, wt_TMPEOTA_load
#     global wt_DEM_load, wt_DBM_load, wt_DOM_load, wt_IPDI_load   
    
        # Add Entry Fields for Material Loadings
    tk.Label(prop_pred, text = 'wt_IPDA loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 3, sticky = W)
    wt_IPDA_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_IPDA_load.grid(column = 1, row = 3, sticky = E)
        # this adds the wt IPDA loading entry field

    
    tk.Label(prop_pred, text = 'wt_HDDA loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 4, sticky = W)
    wt_HDDA_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_HDDA_load.grid(column = 1, row = 4, sticky = E)
        # this adds the wt HDDA loading entry field       
        

    tk.Label(prop_pred, text = 'wt_TMPTA loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 5, sticky = W)
    wt_TMPTA_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_TMPTA_load.grid(column = 1, row = 5, sticky = E)
        # this adds the wt TMPTA loading entry field       
        
    
    tk.Label(prop_pred, text = 'wt_TMPEOTA loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 6, sticky = W)
    wt_TMPEOTA_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_TMPEOTA_load.grid(column = 1, row = 6, sticky = E)
        # this adds the wt TMPEOTA loading entry field  
    

    tk.Label(prop_pred, text = 'wt_DEM loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 7, sticky = W)
    wt_DEM_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_DEM_load.grid(column = 1, row = 7, sticky = E)
        # this adds the wt DEM loading entry field  

        
    tk.Label(prop_pred, text = 'wt_DBM loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 8, sticky = W)
    wt_DBM_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_DBM_load.grid(column = 1, row = 8, sticky = E)
        # this adds the wt DBM loading entry field  

        
    tk.Label(prop_pred, text = 'wt_DOM loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 9, sticky = W)
    wt_DOM_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_DOM_load.grid(column = 1, row = 9, sticky = E)
        # this adds the wt DOM loading entry field  
  

    tk.Label(prop_pred, text = 'wt_IPDI loading',
             background = 'gray92', font = 'Times 13').grid(
             column = 1, row = 10, sticky = W)
    wt_IPDI_load = tk.Entry(prop_pred, textvariable = tk.DoubleVar(prop_pred),
                         font = 'Arial 12')
    wt_IPDI_load.grid(column = 1, row = 10, sticky = E)
        # this adds the wt IPDI loading entry field  


    # Add Notes
    tk.Label(prop_pred, background = 'gray92', text = \
'''  NOTE: All loadings are with respect to total monomer amount.
(i.e. "1" indicates 1% of total monomer weight)''',
             font = 'Times 12', fg = 'red').grid(column = 1,
                                                 rowspan = 2, row = 11)
        # this add notes to be read by the user

    # Add Buttons
    ttk.Button(prop_pred, text = 'TOTAL MONOMER CHECK',
               command = show_monomer).grid(column = 1, row = 13, sticky = W+E)
        # this adds the button to display the total amount of monomer entered

    ttk.Button(prop_pred, text = 'PREDICT', command = predictions).grid(
               column = 1, row = 14, sticky = W+E)
        # this adds the button to display polymer property predictions

    ttk.Button(prop_pred, text = 'QUIT', command = prop_pred.destroy).grid(
               column = 1, row = 15, sticky = W+E)
        # this adds the button to exit the application


    prop_pred.mainloop()


# ''' END OF PROPERTY PREDICTING GUI '''

In [58]:
# CONSTRUCT GUI TO INCLUDE ALL OTHER GUIs

In [61]:
# CONSTRUCT MAIN GUI
main = tk.Tk()
main.title(' MIDAS ')
main.configure(background = 'gray92')

# 'gray20' is nice for dark mode.

for num in range(0,6):
    for col in [0,2]:
        tk.Label(main, text = '', bg = 'gray92').grid(
                 row = num, column = col)
# this loop adds white borders to make the window less busy

axalta = Image.open('axaltalogo.png')
axalta_logo = ImageTk.PhotoImage(axalta)
tk.Label(main, image = axalta_logo).grid(row = 0, column = 0, columnspan = 3)

# this adds the Axalta logo to the window

tk.Label(main, bg = 'gray92', font = 'Helvetica 16', fg ='#000000', text = \
'''MIDAS: Materials Informatics for Designing Aspartate Systems

To begin, select which module you would like to use.''').grid(
         row = 1, column = 1, sticky = W+E)
# this adds a welcome message to the user

# Slimer green is #b7f731

ttk.Style().configure('TButton', font = 'Helvetica 14', fg ='#000000')

ttk.Button(main, text = 'PROPERTY PREDICTOR',
           command = make_property_predictor).grid(
           row = 3, column = 1, sticky = W+E)
# this adds a button to initiate the polymer property predictor

ttk.Button(main, text = 'RECIPE PREDICTOR',
           command = make_recipe_predictor).grid(
           row = 4, column = 1, sticky = W+E)
# this adds a button to initiate the polymer recipe predictor

ttk.Button(main, text = 'QUIT',
           command = main.destroy).grid(
           row = 5, column = 1, sticky = W+E)
# this adds a button to exit the prediction tool


main.mainloop()

In [60]:
# TODO

# Test against target test case
# Extend ranges of input weight fractions
# Add qualitative variables
# Add a way to add data and retrain the model
# Add smaller logos to polymer predictors and property predictors